In [0]:
import requests
import time

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd

In [0]:
def scrape_linkedin(job, city):
    driver = webdriver.Chrome()
    base_url="https://linkedin.com/jobs/search?keywords="
    job=job.replace(" ", "%20").strip()
    city=city.replace(" ", "%20").strip()
    url = f"https://www.linkedin.com/jobs/search?keywords={job}&location={city}"
    driver.get(url)
    counter = 0
    while True:
        try:
            click_me = driver.find_element_by_xpath('/html/body/main/section[1]/button')
            time.sleep(2)
            click_me.click()
            counter += 1
            if counter == 40:
                break
        except Exception as e:
            break
    my_var = driver.find_elements_by_class_name('result-card__full-card-link')
    
    driver2 = webdriver.Chrome()
    position_list = []
    company_list = []
    location_list = []
    description = []
    num_success = 0
    for counter, x in enumerate(my_var):
        if counter % 50 == 0:
            print(f'Trying job number {counter} out of {len(my_var)}')
            print(f'{num_success} out of {counter} successful')
        driver2.get(x.get_attribute('href'))
        time.sleep(1)
        my_html = driver2.page_source
        soup = BeautifulSoup(my_html, 'html.parser')
        soup.find('div', class_ = 'topcard__content-left')
        try:
            position = soup.find('div', class_ = 'topcard__content-left').find('h1', class_ = 'topcard__title').text
        except:
            continue
        job_titles = ['data scientist', 'data engineer', 'data analyst', 'machine learning']
        good_job = False
        for x in job_titles:
            if x in position.lower():
                good_job = True
                break
        if good_job == True:
            try:
                position_list.append(position)
                company = soup.find('div', class_ = 'topcard__content-left').find('a').text
                company_list.append(company)
                location = soup.find('div', class_ = 'topcard__content-left').find('span', class_ = 'topcard__flavor topcard__flavor--bullet').text
                location_list.append(location)
                decr = driver2.find_element_by_class_name('description__text').text
                descr.append(descr)
                num_success += 1
            except AttributeError:
                position_list.pop()
                continue
        time.sleep(2)
    driver.quit()
    driver2.quit()
    df = pd.DataFrame({'position': position_list,
                  'company': company_list,
                  'location': location_list,
                 'description': description})
    time.sleep(60)
    return df

In [0]:
%%time
position = ['data scientist','data engineer', 'data analyst', 'machine learning engineer']
cities = ['chicago, il']
for city in cities:
    for pos in position:
        print(f'Starting {city} {pos}')
        df = scrape_linkedin(pos, city)
        df.to_csv(f'df_{city}_{pos}.csv', index = False)
        print(f'{city} {pos} complete')